# prepare data


In [1]:
import cv2
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import os

#input
test=1
repeat=4

testNo = "test"+str(test)+"."+str(repeat)
flowtestNo = "flow"+str(test)+"."+str(repeat)
folder=["","test1 V0.65 F15","test2 V0.65 F25","test3 V0.65 F35","test4 V0.7 F35","test5 V0.60 F35"]
output_folder = "./asset/"+folder[test]+"/"+testNo
start_time_of_suffution_AllTest=["x",[5,4,2,7],[5,7,3],[10,20,16],[95,7,68],[3,13,12]]


finelossdata =  pd.read_csv(output_folder+'/finepixel '+testNo+' Kmeans.csv')
flowdata = pd.read_csv(output_folder+'/finepixel '+testNo+' flow.csv')
flowdata,finelossdata  = [np.array(flowdata.values), np.array(finelossdata.values)]
dictflowdata = dict(zip(flowdata[:,0], flowdata[:,1]))
# print(dictflowdata)

#add flowdata to finelossdata
processingdata = []
for i in finelossdata:
    if i[0]%1==0 and (i[0] in dictflowdata.keys()):
        processingdata.append(list(i)+[dictflowdata[i[0]]])
    else :
        processingdata.append(list(i)+[""])
processingdata=np.array(processingdata)        
# df = pd.DataFrame(processingdata, columns=['time (s)', 'finepixelkmeans', 'finepixelmask', 'flowrate (L/min)'])
# df.to_csv(output_folder+'/finepixel '+testNo+' sumary.csv', index=False)


len0=processingdata.shape[0]
len1=processingdata.shape[1]
print(len0,len1)
for i in range(len0):
    if (processingdata[i][len1-1])=="":
        if (processingdata[i-1][len1-1])!="" :
            y1=float(processingdata[i-1][len1-1])
            x1=float(processingdata[i-1][0])
            istart=i
            
        if i+1 < len0-1:    #void error
            if (processingdata[i+1][len1-1])!="" :
                y2=float(processingdata[i+1][len1-1])
                x2=float(processingdata[i+1][0])
                m=(y2-y1)/(x2-x1)
                c=y2-(x2*m)
                istop=i+1
                # print(istart,istop)
                # print(x1,x2,y1,y2)
                
                #linear interpolation
                processingdata[istart:istop+1, len1-1]=((processingdata[istart:istop+1, 0]).astype('float64') )*m+c
        if i == len0-1:
            y2=y1
            x2=float(processingdata[i][0])
            m=0
            c=y2-(x2*m)
            istop=i+1
            processingdata[istart:istop+1, len1-1]=((processingdata[istart:istop+1, 0]).astype('float64') )*m+c


# df = pd.DataFrame(processingdata, columns=['time (s)', 'finepixelkmeans', 'finepixelmask', 'flowrate (L/min)'])
# df.to_csv(output_folder+'/finepixel '+testNo+' sumary_linear_interpolation.csv', index=False)


#input boundary condition
sectionarea_MMxMM= 100 * 1.5
sectionarea_MxM=(sectionarea_MMxMM * 10**-6)
dH_CM=10
dL_CM=30
hydraulic_gradient=(dH_CM/dL_CM)

#cal hydraulic conductivity
processingdata[:, len1-1]=( (processingdata[:, len1-1]).astype('float64')/(1000*60) ) / (sectionarea_MxM*hydraulic_gradient)


processingdata=processingdata.astype('float64')
#moving avg 3point
for j in range(5):
    processingdata2=np.array(processingdata)
    for i in range(len0):
        if i>=3-1:
            processingdata2[i ,1:3]=sum(processingdata[(i+1-3):(i+1) ,1:3])/3
    processingdata=np.array(processingdata2)
# use A=np.array(B) instead of A=B
# avoid bug A=B link B to A (B change A will change like excel formular), A=np.array(B) will copy values of B to A

#cal erosion rate
#add new column (erosion rate)
starttime=start_time_of_suffution_AllTest[test][repeat-1]
processingdata=np.append(processingdata, np.zeros((len0,2)), axis=1)
for i in range(len0):
        if i>=starttime :
            y=processingdata[(i-1):(i+1) ,1:3]
            x=processingdata[(i-1):(i+1) ,0:1]
            processingdata[i ,4:6]=((y[1:2,:]-y[0:1,:])/(x[1:2,:]-x[0:1,:]))

# cut initial part
cuttime=starttime
processingdata[:,0]=processingdata[:,0]-cuttime
df = pd.DataFrame(processingdata[cuttime-1:,:], columns=['time (s)', 'finelossML', 'finelossThreshold', 'K (m/s)', 'erosionRateML','erosionRateThreshold'])
df.to_csv(output_folder+'/finepixel '+testNo+' final.csv', index=False)


9369 4


# linear interpolation for flow (run this when error nan)

In [ ]:
import cv2
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import os

#input
test=3
repeat=6

testNo = "test"+str(test)+"."+str(repeat)
folder=["","test1 V0.65 F15","test2 V0.65 F25","test3 V0.65 F35","test4 V0.7 F35","test5 V0.60 F35"]
output_folder = "./asset/"+folder[test]+"/"+testNo


flowdata = pd.read_csv(output_folder+'/finepixel '+testNo+' flow.csv')
processingdata=np.array(flowdata)
processingdata = np.where(np.isnan(processingdata),"", processingdata)
len0=processingdata.shape[0]
len1=processingdata.shape[1]
print(len0,len1)
# print(processingdata)
for i in range(len0):
    if (processingdata[i][len1-1])=="":
        if (processingdata[i-1][len1-1])!="" :
            y1=float(processingdata[i-1][len1-1])
            x1=float(processingdata[i-1][0])
            istart=i
            
        if i+1 < len0-1:    #void error
            if (processingdata[i+1][len1-1])!="" :
                y2=float(processingdata[i+1][len1-1])
                x2=float(processingdata[i+1][0])
                m=(y2-y1)/(x2-x1)
                c=y2-(x2*m)
                istop=i+1
                # print(istart,istop)
                # print(x1,x2,y1,y2)
                
                #linear interpolation
                processingdata[istart:istop+1, len1-1]=((processingdata[istart:istop+1, 0]).astype('float64') )*m+c
        if i == len0-1:
            y2=y1
            x2=float(processingdata[i][0])
            m=0
            c=y2-(x2*m)
            istop=i+1
            processingdata[istart:istop+1, len1-1]=((processingdata[istart:istop+1, 0]).astype('float64') )*m+c

df = pd.DataFrame(processingdata, columns=['time (s)', 'flowrate (L/min)'])
df.to_csv(output_folder+'/finepixel '+testNo+' flow.csv', index=False)


In [ ]:
["a"][""]*8+["a"]

In [ ]:
d={1: "Tom", "age": 10}
x=1
d.keys()

In [ ]:
68.0%1

In [ ]:
(processingdata[13:73, 0])

In [ ]:
import numpy as np
NP=np.array([[1,3,6],
             [1.5,4,9]])
lenNP0=NP.shape[0]
lenNP1=NP.shape[1]
print(lenNP0)
print(NP[:,1:3])
print(sum(NP[:,0:1]))
y=NP[:,1:3]
x=NP[:,0:1]
print((y[1:2,:]-y[0:1,:]))
print((x[1:2,:]-x[0:1,:]))
print((y[1:2,:]-y[0:1,:])/(x[1:2,:]-x[0:1,:]))
a=np.zeros((2,2))
# a=NP[:,0:2]
print(a)
NP=(np.append(NP,a, axis=1))
print(NP)
NP2=np.zeros((2,5))
NP2[0,:]=sum(NP[:,:])/2
NP2[1,:]=sum(NP[:,:])/3
print(NP2)
print(NP)
NP=np.array(NP2)
NP2[0,:]=sum(NP[:,:])/2
NP2[1,:]=sum(NP[:,:])/3
print(NP2)
print(NP)


In [ ]:
10**-3*10**-3

In [ ]:
processingdata[20][1]

In [ ]:
if "1"%0==0 :
    print("h")